- 5/12일 기준 최고기록은 라벨인코딩을 한번에 해준 것

In [38]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action = 'ignore')

In [52]:
## 데이터 불러오기
trn = pd.read_csv('data/train.csv')
tst = pd.read_csv('data/test.csv')

In [53]:
## 365 변환
def days_to_plus(x):
    return (x*-1)/365

## 업무 시작일 변환
def plus(x):
    if x<0:
        return x*(-1)
    else:
        return 0

## 나이 양수로 변환
trn['birth_plus'] = trn.DAYS_BIRTH.map(days_to_plus)
tst['birth_plus'] = tst.DAYS_BIRTH.map(days_to_plus)
    
## 업무시작일 양수로 변환
trn['employed_plus'] = trn.DAYS_EMPLOYED.map(days_to_plus)
tst['employed_plus'] = tst.DAYS_EMPLOYED.map(days_to_plus)

In [26]:
# ## incoome total 내림차순으로 정렬 및 인코딩
# lbe = LabelEncoder()
# lbe.fit(list(trn.groupby('edu_type')['income_total'].mean().sort_values(ascending = False).index))
# trn.edu_type = lbe.transform(trn.edu_type)
# tst.edu_type = lbe.transform(tst.edu_type)

In [27]:
# ## incoome total 내림차순으로 정렬 및 인코딩
# lbe = LabelEncoder()
# lbe.fit(list(trn.groupby('house_type')['income_total'].mean().sort_values(ascending = False).index))
# trn.house_type = lbe.transform(trn.house_type)
# tst.house_type = lbe.transform(tst.house_type)

In [41]:
# ## occyp_type의 결측값들을 NONE이라는 범주로 만듦
# trn['occyp_type'] = trn['occyp_type'].fillna('NONE').astype('object').values
# tst['occyp_type'] = tst['occyp_type'].fillna('NONE').astype('object').values

In [29]:
# ## incoome total 내림차순으로 정렬 및 인코딩
# lbe = LabelEncoder()
# lbe.fit(list(trn.groupby('occyp_type')['income_total'].mean().sort_values(ascending = False).index))
# trn.occyp_type = lbe.transform(trn.occyp_type)
# tst.occyp_type = lbe.transform(tst.occyp_type)

In [30]:
# ## incoome total 내림차순으로 정렬 및 인코딩
# lbe = LabelEncoder()
# lbe.fit(list(trn.groupby('income_type')['income_total'].mean().sort_values(ascending = False).index))
# trn.income_type = lbe.transform(trn.income_type)
# tst.income_type = lbe.transform(tst.income_type)

In [54]:
index_col = 'index'
target_col = 'credit'

cat_cols = [x for x in trn.columns if trn[x].dtype == 'object']
float_cols = [x for x in trn.columns.drop('credit') if trn[x].dtype == 'float64']
num_cols = [x for x in trn.columns if x not in cat_cols + [target_col]]
feature_cols = num_cols + cat_cols
print(len(feature_cols), len(cat_cols), len(num_cols))

21 8 13


In [55]:
from sklearn.preprocessing import LabelEncoder

lbe = LabelEncoder()
for i in cat_cols:
    trn[i] = lbe.fit_transform(trn[i])
    tst[i] = lbe.transform(tst[i])
    
for i in float_cols:
    trn[i] = trn[i].astype('int')
    tst[i] = tst[i].astype('int')

In [56]:
## 인덱스 설정
trn.set_index('index', inplace = True)
tst.set_index('index', inplace = True)

In [57]:
trn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26457 entries, 0 to 26456
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         26457 non-null  int32  
 1   car            26457 non-null  int32  
 2   reality        26457 non-null  int32  
 3   child_num      26457 non-null  int64  
 4   income_total   26457 non-null  int32  
 5   income_type    26457 non-null  int32  
 6   edu_type       26457 non-null  int32  
 7   family_type    26457 non-null  int32  
 8   house_type     26457 non-null  int32  
 9   DAYS_BIRTH     26457 non-null  int64  
 10  DAYS_EMPLOYED  26457 non-null  int64  
 11  FLAG_MOBIL     26457 non-null  int64  
 12  work_phone     26457 non-null  int64  
 13  phone          26457 non-null  int64  
 14  email          26457 non-null  int64  
 15  occyp_type     26457 non-null  int32  
 16  family_size    26457 non-null  int32  
 17  begin_month    26457 non-null  int32  
 18  credit

In [58]:
trn.credit = trn.credit.astype('category')

In [59]:
trn1 = trn.drop('family_size', axis = 1).copy()
trn2 = trn.drop('child_num', axis = 1).copy()
trn3 = trn.drop('birth_plus', axis = 1).copy()
trn4 = trn.drop('employed_plus', axis = 1).copy()
trn5 = trn.drop(['birth_plus', 'employed_plus'], axis = 1).copy()
# 제일 좋음
trn6 = trn.drop(['family_size', 'employed_plus'], axis = 1).copy()
trn7 = trn.drop(['family_size', 'birth_plus', 'employed_plus'], axis = 1).copy()

In [ ]:
# train과 valid(test) 셋을 분리 
x_train, x_valid, y_train, y_valid = train_test_split(trn6.drop('credit', axis=1), trn.credit, test_size=0.3, stratify=trn.credit, random_state=40)

# RF 모델링 
rf = RandomForestClassifier(n_estimators = 5000,
                            max_features = 3,
                            min_samples_split = 6,
                            random_state = 2021,
                            n_jobs = 10
                           )
rf.fit(x_train, y_train)
rf_pred = rf.predict_proba(x_valid)
log_loss(y_valid, rf_pred)

In [ ]:
### 이 기록은 라벨인코딩을 따로 지정하지 않고 한번에 돌린 데이터
### trn6은 trn6 = trn.drop(['family_size', 'employed_plus'], axis = 1).copy()이다.
### 총 18개의 변수를 사용한 것 -> 최고기록

## predict_rf
ftr = trn6.drop('credit', axis=1).values
target = trn['credit'].values
tst_ar = tst.drop(['family_size', 'employed_plus'], axis = 1).values
n_class = 3
n_fold = 5

print(ftr.shape)
print(tst_ar.shape)


cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=40)

rf_p_val = np.zeros((ftr.shape[0], n_class))
rf_p_tst = np.zeros((tst_ar.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    rf_clf = RandomForestClassifier(n_estimators = 5000, 
                                random_state=2021,
                                max_features = 3,
                                min_samples_split = 6,
                                n_jobs=-1)
    rf_clf.fit(ftr[i_trn], target[i_trn])
    rf_p_val[i_val, :] = rf_clf.predict_proba(ftr[i_val])
    rf_p_tst += rf_clf.predict_proba(tst_ar) / n_fold
    
print(f'{log_loss(target, rf_p_val)}')
print(f'{confusion_matrix(target, np.argmax(rf_p_val, axis=1))}%')

- 0.7181191299537627
: 3fold
[[  564   510  2148]
 [  137  2700  3430]
 [  337   865 15766]]%
     rf_clf = RandomForestClassifier(n_estimators = 5000, 
                                random_state=2021,
                                max_features = 3,
                                min_samples_split = 6,
                                n_jobs=6)
                                
- 0.7027768293407574
: 5fold
0.7027768293407574
[[  656   527  2039]
 [  149  2864  3254]
 [  355   899 15714]]%
     rf_clf = RandomForestClassifier(n_estimators = 5000, 
                                random_state=2021,
                                max_features = 3,
                                min_samples_split = 6,
                                n_jobs=6)

## trn6에서 cv = 10으로 해보기
- 벨리드 성능 먼저 확인 : 0.7029055255003107 보다 높아야 함
- 0.694738615715623 나옴

In [60]:
### 이 기록은 라벨인코딩을 따로 지정하지 않고 한번에 돌린 데이터
### trn6은 trn6 = trn.drop(['family_size', 'employed_plus'], axis = 1).copy()이다.
### 총 18개의 변수를 사용한 것 -> 최고기록

## predict_rf
ftr = trn6.drop('credit', axis=1).values
target = trn['credit'].values
tst_ar = tst.drop(['family_size', 'employed_plus'], axis = 1).values
n_class = 3
n_fold = 10

print(ftr.shape)
print(tst_ar.shape)


cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

rf_p_val = np.zeros((ftr.shape[0], n_class))
rf_p_tst = np.zeros((tst_ar.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    rf_clf = RandomForestClassifier(n_estimators = 5000, 
                                random_state=2021,
                                max_features = 3,
                                min_samples_split = 6,
                                n_jobs=1)
    rf_clf.fit(ftr[i_trn], target[i_trn])
    rf_p_val[i_val, :] = rf_clf.predict_proba(ftr[i_val])
    rf_p_tst += rf_clf.predict_proba(tst_ar) / n_fold
    
print(f'{log_loss(target, rf_p_val)}')
print(f'{confusion_matrix(target, np.argmax(rf_p_val, axis=1))}%')

(26457, 18)
(10000, 18)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5
training model for CV #6
training model for CV #7
training model for CV #8
training model for CV #9
training model for CV #10
0.694738615715623
[[  677   513  2032]
 [  153  2890  3224]
 [  368   867 15733]]%


In [61]:
sub = pd.read_csv('data/sample_submission.csv', index_col = 0)
sub
sub[sub.columns] = rf_p_tst

In [62]:
sub.to_csv('data/최고기록 CV=10.csv')

## 연봉을 추가한 cv10

In [63]:
trn6['income_total_2'] = trn6.income_total**2
tst['income_total_2'] = tst.income_total**2

In [65]:
### 이 기록은 라벨인코딩을 따로 지정하지 않고 한번에 돌린 데이터
### trn6은 trn6 = trn.drop(['family_size', 'employed_plus'], axis = 1).copy()이다.
### 총 18개의 변수를 사용한 것 -> 최고기록

## predict_rf
ftr = trn6.drop('credit', axis=1).values
target = trn['credit'].values
tst_ar = tst.drop(['family_size', 'employed_plus'], axis = 1).values
n_class = 3
n_fold = 10

print(ftr.shape)
print(tst_ar.shape)


cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

rf_p_val = np.zeros((ftr.shape[0], n_class))
rf_p_tst = np.zeros((tst_ar.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    rf_clf = RandomForestClassifier(n_estimators = 5000, 
                                random_state=2021,
                                max_features = 3,
                                min_samples_split = 6,
                                n_jobs=6)
    rf_clf.fit(ftr[i_trn], target[i_trn])
    rf_p_val[i_val, :] = rf_clf.predict_proba(ftr[i_val])
    rf_p_tst += rf_clf.predict_proba(tst_ar) / n_fold
    
print(f'{log_loss(target, rf_p_val)}')
print(f'{confusion_matrix(target, np.argmax(rf_p_val, axis=1))}%')

(26457, 19)
(10000, 19)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5
training model for CV #6
training model for CV #7
training model for CV #8
training model for CV #9
training model for CV #10
0.6933945398659884
[[  708   521  1993]
 [  156  2942  3169]
 [  386   901 15681]]%


In [69]:
sub = pd.read_csv('data/sample_submission.csv', index_col = 0)
sub
sub[sub.columns] = rf_p_tst

In [70]:
sub.to_csv('data/최고기록+연봉weight CV=10.csv')

-----

## trn6 + 연봉 변수 제곱 추가 해보기
- 벨리드 성능 확인 : 0.7029055255003107 보다 높아야 함
- 처음에는 5폴드로 진행하기(R 베이스 라인과 파이썬의 차이에서 진행중) : 
- 이후 10폴드 진행 : 


In [ ]:
trn6['income_total_2'] = trn6.income_total**2
tst['income_total_2'] = tst.income_total**2

In [ ]:
### 이 기록은 라벨인코딩을 따로 지정하지 않고 한번에 돌린 데이터
### trn6은 trn6 = trn.drop(['family_size', 'employed_plus'], axis = 1).copy()이다.
### 총 18개의 변수를 사용한 것 -> 최고기록

## predict_rf
ftr = trn6.drop('credit', axis=1).values
target = trn['credit'].values
tst_ar = tst.drop(['family_size', 'employed_plus'], axis = 1).values
n_class = 3
n_fold = 5

print(ftr.shape)
print(tst_ar.shape)


cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

rf_p_val = np.zeros((ftr.shape[0], n_class))
rf_p_tst = np.zeros((tst_ar.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    rf_clf = RandomForestClassifier(n_estimators = 5000, 
                                random_state=2021,
                                max_features = 3,
                                min_samples_split = 6,
                                n_jobs=-1)
    rf_clf.fit(ftr[i_trn], target[i_trn])
    rf_p_val[i_val, :] = rf_clf.predict_proba(ftr[i_val])
    rf_p_tst += rf_clf.predict_proba(tst_ar) / n_fold
    
print(f'{log_loss(target, rf_p_val)}')
print(f'{confusion_matrix(target, np.argmax(rf_p_val, axis=1))}%')

------

## Extra Tree 확인
- trn6를 extra tree에 넣어서 확인
- 그리드 서치를 통해 찾은 파라미터를 사용

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
param_grid = {
    'n_estimators': [1000, 2000, 3000, 4000],
    'max_features': [3, 4, 5, 6, 7, 8, 9],
    'min_samples_split': [4,5,6,7,8,9,10],
    'random_state' : [2021]
}

In [ ]:
estimator = ExtraTreesClassifier()

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=40)

In [ ]:
# define grid_search
grid_search = GridSearchCV(estimator=estimator, 
                           param_grid=param_grid, 
                           cv=kf, 
                           n_jobs=2, 
                           verbose=2
                          )

# fit with (x_train, y_train)
grid_search.fit(x_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
## predict_rf
ftr = trn6.drop('credit', axis=1).values
target = trn['credit'].values
tst_ar = tst.drop(['family_size', 'employed_plus'], axis = 1).values
n_class = 3
n_fold = 10

print(ftr.shape)
print(tst_ar.shape)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print(ftr.shape)

ext_p_val = np.zeros((ftr.shape[0], n_class))
ext_p_tst = np.zeros((tst_ar.shape[0], n_class))

for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    ext_clf = ExtraTreesClassifier(n_estimators=5000, 
                                   random_state=2021)
    
    ext_clf.fit(ftr[i_trn], tst_ar[i_trn])
    ext_p_val[i_val, :] = ext_clf.predict_proba(ftr[i_val])
    ext_p_tst += ext_clf.predict_proba(tst_ar) / n_fold
    
    print(log_loss(target[i_val],np.argmax(ext_p_val[i_val],axis=1)))

print(f'{log_loss(target, ext_p_val)}')
print(confusion_matrix(target, np.argmax(ext_p_val, axis=1)))